# CNN
卷积神经网络（convolutional neural network）是含有卷积层（convolutional layer）的神经网络。本章中介绍的卷积神经网络均使用最常见的二维卷积层。它有高和宽两个空间维度，常用来处理图像数据。

In [1]:
import torch
import torch.nn as nn

先计算 应用 滤波器 f 应用于 X矩阵 后， 产生的输出的大小h, w
然后分别滑动窗口计算 每一个patch 的输出，sum()

In [27]:
import torch
import torch.nn as nn

def corr2d(X, kernel):
    h, w = kernel.shape
    out = torch.zeros(X.size()[0] - h + 1, X.size()[1] - w + 1 )
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
            out[i, j] = (X[i : i+h, j : j+w] *kernel).sum()
    return out
    
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [1, 0]])



In [28]:
cov2d(X,K)

tensor([[ 4.,  6.],
        [10., 12.]])

 ## 二维卷积层 Pytorch
 
 
 卷积窗口形状为$p \times q$的卷积层称为$p \times q$卷积层。同样，$p \times q$卷积或$p \times q$卷积核说明卷积核的高和宽分别为$p$和$q$。

In [29]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张$6\times 8$的图像（即高和宽分别为6像素和8像素的图像）。它中间4列为黑（0），其余为白（1）。

In [37]:
image = torch.ones(6,8)
image[:,2:6] = 0
print(image.dtype)

torch.float32


In [38]:
from PIL import Image
import numpy as np
def show_numpy_GRAY(image):
    data = image.numpy().astype(np.uint8)
    data *= 255
    img = Image.fromarray(data, 'L')
    img.show()
show_numpy_GRAY(image)

In [39]:
K = torch.tensor([[1.0, -1]])
Y = corr2d(image, K)
show_numpy_GRAY(Y)

## 通过数据学习核数组

最后我们来看一个例子，它使用物体边缘检测中的输入数据`X`和输出数据`Y`来学习我们构造的核数组`K`。我们首先构造一个卷积层，其卷积核将被初始化成随机数组。接下来在每一次迭代中，我们使用平方误差来比较`Y`和卷积层的输出，然后计算梯度来更新权重。

In [43]:
# 构造一个核数组形状是(1, 2)的二维卷积层
CONV2D = Conv2D(kernel_size=(1, 2))
K = torch.tensor([[1, -1]], dtype = torch.float)
print(K.dtype)
Y = corr2d(image, K)

step = 30
lr = 0.01

print("weight: ", CONV2D.weight.data)
print("bias: ", CONV2D.bias.data)

for i in range(step):
    Y_hat = CONV2D(image)
    
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()
    
    CONV2D.weight.data -= lr * CONV2D.weight.grad
    CONV2D.bias.data -= lr * CONV2D.bias.grad
    
    # 梯度清0
    CONV2D.weight.grad.fill_(0)
    CONV2D.bias.grad.fill_(0)
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))
        
print("weight: ", CONV2D.weight.data)
print("bias: ", CONV2D.bias.data)

torch.float32
weight:  tensor([[ 1.3820, -0.5599]])
bias:  tensor([-0.1046])
Step 5, loss 0.466
Step 10, loss 0.053
Step 15, loss 0.006
Step 20, loss 0.001
Step 25, loss 0.000
Step 30, loss 0.000
weight:  tensor([[ 0.9998, -0.9990]])
bias:  tensor([-0.0004])


可以看到，学到的卷积核的权重参数与我们之前定义的核数组`K`较接近，而偏置参数接近0。

In [44]:
print("weight: ", CONV2D.weight.data)
print("bias: ", CONV2D.bias.data)

weight:  tensor([[ 0.9998, -0.9990]])
bias:  tensor([-0.0004])


# 互相关运算和卷积运算
## 标准的卷积操作是从右下角开始，从右向左，从下到上，依次相乘。

## 深度学习中的卷积，其实是互相关，是直观的对应相乘求和操作。

## 互相关运算和卷积运算

实际上，卷积运算与互相关运算类似。**为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输入数组做互相关运算**。可见，卷积运算和互相关运算虽然类似，但如果它们使用相同的核数组，对于同一个输入，输出往往并不相同。

那么，你也许会好奇卷积层为何能使用互相关运算替代卷积运算。其实，在深度学习中核数组都是学出来的：卷积层无论使用互相关运算或卷积运算都不影响模型预测时的输出。为了解释这一点，假设卷积层使用互相关运算学出图5.1中的核数组。设其他条件不变，使用卷积运算学出的核数组即图5.1中的核数组按上下、左右翻转。也就是说，图5.1中的输入与学出的已翻转的核数组再做卷积运算时，依然得到图5.1中的输出。为了与大多数深度学习文献一致，如无特别说明，本书中提到的卷积运算均指互相关运算。

> 注：感觉深度学习中的卷积运算实际上是互相关运算是个面试题考点。

## 5.1.6 特征图和感受野

二维卷积层输出的二维数组可以看作是输入在空间维度（宽和高）上某一级的表征，也叫特征图（feature map）。影响元素$x$的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做$x$的感受野（receptive field）。以图5.1为例，输入中阴影部分的四个元素是输出中阴影部分元素的感受野。我们将图5.1中形状为$2 \times 2$的输出记为$Y$，并考虑一个更深的卷积神经网络：将$Y$与另一个形状为$2 \times 2$的核数组做互相关运算，输出单个元素$z$。那么，$z$在$Y$上的感受野包括$Y$的全部四个元素，在输入上的感受野包括其中全部9个元素。可见，我们可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征。

我们常使用“元素”一词来描述数组或矩阵中的成员。在神经网络的术语中，这些元素也可称为“单元”。当含义明确时，本书不对这两个术语做严格区分。

## 小结

- 二维卷积层的核心计算是二维互相关运算。在最简单的形式下，它对二维输入数据和卷积核做互相关运算然后加上偏差。
- 我们可以设计卷积核来检测图像中的边缘。
- 我们可以通过数据来学习卷积核。

# 填充和步幅

在上一节的例子里，我们使用高和宽为3的输入与高和宽为2的卷积核得到高和宽为2的输出。一般来说，假设输入形状是$n_h\times n_w$，卷积核窗口形状是$k_h\times k_w$，那么输出形状将会是

$$(n_h-k_h+1) \times (n_w-k_w+1).$$

所以卷积层的输出形状由输入形状和卷积核窗口形状决定。本节我们将介绍卷积层的两个超参数，即填充和步幅。它们可以对给定形状的输入和卷积核改变输出形状。

In [47]:
X=torch.randn(8,8)
print(X.shape)

torch.Size([8, 8])


In [48]:
(1,1)+X.shape

(1, 1, 8, 8)

In [55]:
# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
# 是不行的，X 是一个元素，8*8
# pytorch 的 输出必须是一个四维 tensor batch_size, channels, H, W
X=torch.randn(8,8)
X = X.view((1,1)+X.shape)
Y = conv2d(X)
Y.shape[2:]

torch.Size([8, 8])

In [54]:
Y.shape[2:]

torch.Size([8, 8])

## 步幅

在上一节里我们介绍了二维互相关运算。卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。我们将每次滑动的行数和列数称为步幅（stride）。

一般来说，当高上步幅为$s_h$，宽上步幅为$s_w$时，输出形状为

$$\lfloor(n_h-k_h+2*p_h+s_h  )/s_h\rfloor \times \lfloor(n_w-k_w+ 2*p_w+s_w )/s_w\rfloor.$$

如果设置$p_h=k_h-1$和$p_w=k_w-1$，那么输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是$(n_h/s_h) \times (n_w/s_w)$。

In [118]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)

X=torch.randn(8,8)
X = X.view((1,1)+X.shape)
Y = conv2d(X)
Y.shape[2:]

torch.Size([4, 4])

计算过程
$$
{(h - k_h + 2*padding_h   + stride_h )/ stride_h}  \times {(w - k_w+ 2*padding_w  +  stride_w )/ stride_w }
$$

$$
{(8-3+1*2+2)/2} \times {(8-3+1*2+2)/2} = 4\times 4
$$

In [64]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 5), padding=(0, 1), stride=(3, 4))

X=torch.randn(8,8)
X = X.view((1,1)+X.shape)
Y = conv2d(X)
Y.shape[2:]

#当计算公式为小数时，向下取整，因为最后分数次是无法卷积的/

torch.Size([2, 2])

$$
{(8-5+2*0 +3)/3} \times {(8-5+2*1+4)/4} = 2\times 2
$$

为了表述简洁，当输入的高和宽两侧的填充数分别为$p_h$和$p_w$时，我们称填充为$(p_h, p_w)$。特别地，当$p_h = p_w = p$时，填充为$p$。当在高和宽上的步幅分别为$s_h$和$s_w$时，我们称步幅为$(s_h, s_w)$。特别地，当$s_h = s_w = s$时，步幅为$s$。在默认情况下，填充为0，步幅为1。

# 多通道
## 多通道输入

In [76]:
import torch
from torch import nn


def corr2d_multi_in(X, K):
    # 沿着X和K的第0维（通道维）分别计算再相加
    print(X[0, :, :], K[0, :, :])
    res = corr2d(X[0, :, :], K[0, :, :])
    #接下来我们实现含多个输入通道的互相关运算。我们只需要对每个通道做互相关运算，然后进行累加。
    for i in range(1, X.shape[0]):
        print(X[i, :, :], K[i, :, :])
        res += corr2d(X[i, :, :], K[i, :, :])
    return res

X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])

K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]) tensor([[0, 1],
        [2, 3]])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]) tensor([[1, 2],
        [3, 4]])


tensor([[ 56.,  72.],
        [104., 120.]])

## 多通道输出

torch.stack 堆叠 增加新的维度进行堆叠

In [93]:
data1 = torch.randn(3,4)
print(data1)
data2 = torch.randn(3,4)
print(data2)

# 增加一个维度， 两个tensor 堆叠
print(torch.stack([data1,data2]))

print(torch.stack([data1,data2], dim =1))

print(torch.stack([data1,data2], dim =2))


tensor([[-0.2343,  0.5712, -2.2741, -0.5701],
        [-1.4972,  0.8748, -0.2513,  0.4993],
        [-0.5000, -0.0360,  0.2205,  1.9506]])
tensor([[ 0.3481, -0.6405, -0.3464, -0.6964],
        [-1.4464, -1.1613, -0.2279,  0.9000],
        [-1.1952,  2.0768,  0.3028, -0.3127]])
tensor([[[-0.2343,  0.5712, -2.2741, -0.5701],
         [-1.4972,  0.8748, -0.2513,  0.4993],
         [-0.5000, -0.0360,  0.2205,  1.9506]],

        [[ 0.3481, -0.6405, -0.3464, -0.6964],
         [-1.4464, -1.1613, -0.2279,  0.9000],
         [-1.1952,  2.0768,  0.3028, -0.3127]]])
tensor([[[-0.2343,  0.5712, -2.2741, -0.5701],
         [ 0.3481, -0.6405, -0.3464, -0.6964]],

        [[-1.4972,  0.8748, -0.2513,  0.4993],
         [-1.4464, -1.1613, -0.2279,  0.9000]],

        [[-0.5000, -0.0360,  0.2205,  1.9506],
         [-1.1952,  2.0768,  0.3028, -0.3127]]])
tensor([[[-0.2343,  0.3481],
         [ 0.5712, -0.6405],
         [-2.2741, -0.3464],
         [-0.5701, -0.6964]],

        [[-1.4972, -1.4464],
 

In [77]:
K = torch.stack([K, K + 1, K + 2])
def coor2d_muliti_in_out(X,K):
    out = []
    for k in K:
        out.append(corr2d_multi_in(X,k))
        #stack 将数组 合并 按维度
    return torch.stack(out)



print(K.shape) # torch.Size([3, 2, 2, 2])


out = coor2d_muliti_in_out(X, K)
print(out)

torch.Size([3, 2, 2, 2])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]) tensor([[0, 1],
        [2, 3]])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]) tensor([[1, 2],
        [3, 4]])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]) tensor([[1, 2],
        [3, 4]])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]) tensor([[2, 3],
        [4, 5]])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]) tensor([[2, 3],
        [4, 5]])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]) tensor([[3, 4],
        [5, 6]])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


## 1X1 卷积

### 其实可以考虑为一个信道压缩的方式， 输出大小不变，通道减少，将一个通道看作一个特征，然后进行压缩？
最后我们讨论卷积窗口形状为$1\times 1$（$k_h=k_w=1$）的多通道卷积层。我们通常称之为$1\times 1$卷积层，并将其中的卷积运算称为$1\times 1$卷积。因为使用了最小窗口，$1\times 1$卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。实际上，$1\times 1$卷积的主要计算发生在通道维上。图5.5展示了使用输入通道数为3、输出通道数为2的$1\times 1$卷积核的互相关计算。值得注意的是，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，**那么$1\times 1$卷积层的作用与全连接层等价**。

## 小结

- 使用多通道可以拓展卷积层的模型参数。
- 假设将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。
- $1\times 1$卷积层通常用来调整网络层之间的通道数，并控制模型复杂度。

# 池化

In [108]:
import torch
from torch import nn

def pool2d(X, pool_size, mode='max'):
    X = X.float()
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()       
    return Y

In [95]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [96]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## 填充和步幅

同卷积层一样，池化层也可以在输入的高和宽两侧的填充并调整窗口的移动步幅来改变输出形状。池化层填充和步幅与卷积层填充和步幅的工作机制一样。我们将通过`nn`模块里的二维最大池化层`MaxPool2d`来演示池化层填充和步幅的工作机制。我们先构造一个形状为(1, 1, 4, 4)的输入数据，前两个维度分别是批量和通道。

In [101]:
X = torch.arange(16, dtype=torch.float).view((1, 1, 4, 4))

In [102]:
print(X)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])


默认情况下，`MaxPool2d`实例里步幅和池化窗口形状相同。下面使用形状为(3, 3)的池化窗口，默认获得形状为(3, 3)的步幅。

In [109]:
print(pool2d(X.view(-1,4),(2,2)))
pool2d = nn.MaxPool2d(3)
print(pool2d(X)) 

tensor([[ 5.,  6.,  7.],
        [ 9., 10., 11.],
        [13., 14., 15.]])
tensor([[[[10.]]]])


In [110]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [111]:
pool2d = nn.MaxPool2d((2, 4), padding=(1, 2), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

##  多通道

在处理多通道输入数据时，**池化层对每个输入通道分别池化，而不是像卷积层那样将各通道的输入按通道相加**。这意味着池化层的输出通道数与输入通道数相等。下面将数组`X`和`X+1`在通道维上连结来构造通道数为2的输入。

In [112]:
X = torch.cat((X, X + 1), dim=1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [113]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

## 小结

- 最大池化和平均池化分别取池化窗口中输入元素的最大值和平均值作为输出。
- 池化层的一个主要作用是缓解卷积层对位置的过度敏感性。
- 可以指定池化层的填充和步幅。
- 池化层的输出通道数跟输入通道数相同。